In [3]:
# Import required libraries
import pandas as pd
import sqlite3
from datetime import datetime
from abc import ABC, abstractmethod
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from typing import List, Dict, Optional
import hashlib
import re

# ============================================
# DATABASE SETUP
# ============================================

def create_database():
    """Create SQLite database and tables"""
    conn = sqlite3.connect('tax_arrears.db')
    cursor = conn.cursor()
    
    # Create Taxpayers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS taxpayers (
            pin_no TEXT PRIMARY KEY,
            tax_payer_name TEXT NOT NULL,
            phone_no TEXT,
            station_name TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Create Tax Records table (renamed from tax_obligations)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tax_records (
            record_id INTEGER PRIMARY KEY AUTOINCREMENT,
            pin_no TEXT,
            obligation_name TEXT NOT NULL,
            from_date DATE,
            to_date DATE,
            principal REAL DEFAULT 0,
            penalty REAL DEFAULT 0,
            interest REAL DEFAULT 0,
            grand_total REAL DEFAULT 0,
            status TEXT DEFAULT 'PENDING',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (pin_no) REFERENCES taxpayers(pin_no)
        )
    ''')
    
    # Create Audit Log table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS audit_log (
            log_id INTEGER PRIMARY KEY AUTOINCREMENT,
            action TEXT,
            pin_no TEXT,
            record_id INTEGER,
            old_value TEXT,
            new_value TEXT,
            changed_by TEXT DEFAULT 'SYSTEM',
            changed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Create Users table for authentication
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password_hash TEXT NOT NULL,
            role TEXT DEFAULT 'VIEWER',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()
    print("Database created successfully!")

# ============================================
# OOP CLASSES
# ============================================

class Taxpayer:
    """Represents a taxpayer entity"""
    
    def __init__(self, pin_no: str, tax_payer_name: str, phone_no: str = None, station_name: str = None):
        self._pin_no = pin_no
        self._tax_payer_name = tax_payer_name
        self._phone_no = self._clean_phone(phone_no) if phone_no else None
        self._station_name = station_name
        self._records = []  # Renamed from _obligations
    
    def _clean_phone(self, phone: str) -> str:
        """Clean phone number by removing spaces and special characters"""
        if phone:
            return re.sub(r'[^\d+]', '', str(phone))
        return None
    
    # Properties with getters
    @property
    def pin_no(self):
        return self._pin_no
    
    @property
    def tax_payer_name(self):
        return self._tax_payer_name
    
    @property
    def phone_no(self):
        return self._phone_no
    
    @property
    def station_name(self):
        return self._station_name
    
    def add_record(self, record):  # Renamed from add_obligation
        """Add a tax record to this taxpayer"""
        self._records.append(record)
    
    def get_total_arrears(self) -> float:
        """Calculate total arrears for this taxpayer"""
        return sum(record.grand_total for record in self._records)
    
    def get_outstanding_balance(self) -> float:
        """Calculate outstanding balance"""
        total_principal = sum(record.principal for record in self._records)
        return total_principal
    
    def to_dict(self) -> Dict:
        """Convert taxpayer object to dictionary"""
        return {
            'pin_no': self._pin_no,
            'tax_payer_name': self._tax_payer_name,
            'phone_no': self._phone_no,
            'station_name': self._station_name
        }


class TaxRecord:  # Renamed from TaxObligation
    """Represents a tax record entity"""
    
    def __init__(self, pin_no: str, obligation_name: str, from_date: str = None,
                 to_date: str = None, principal: float = 0, penalty: float = 0,
                 interest: float = 0):
        self._pin_no = pin_no
        self._obligation_name = obligation_name
        self._from_date = from_date
        self._to_date = to_date
        self._principal = self._clean_amount(principal)
        self._penalty = self._clean_amount(penalty)
        self._interest = self._clean_amount(interest)
        self._grand_total = self._principal + self._penalty + self._interest
        self._status = self._determine_status()
    
    def _clean_amount(self, amount) -> float:
        """Clean amount by removing commas and converting to float"""
        if isinstance(amount, str):
            amount = amount.replace(',', '').replace('"', '').strip()
            if amount in ['-', '']:
                return 0.0
        try:
            return float(amount)
        except (ValueError, TypeError):
            return 0.0
    
    @property
    def grand_total(self):
        return self._grand_total
    
    @property
    def principal(self):
        return self._principal
    
    @property
    def obligation_name(self):
        return self._obligation_name
    
    def _determine_status(self) -> str:
        """Determine the status of the record"""
        if self._grand_total <= 0:
            return "PAID"
        elif self._penalty > 0 or self._interest > 0:
            return "PENALTY_APPLIED"
        else:
            return "PENDING"
    
    def to_dict(self) -> Dict:
        """Convert record object to dictionary"""
        return {
            'pin_no': self._pin_no,
            'obligation_name': self._obligation_name,
            'from_date': self._from_date,
            'to_date': self._to_date,
            'principal': self._principal,
            'penalty': self._penalty,
            'interest': self._interest,
            'grand_total': self._grand_total,
            'status': self._status
        }


class DatabaseManager:
    """Handles all database operations"""
    
    def __init__(self, db_path='tax_arrears.db'):
        self.db_path = db_path
    
    def get_connection(self):
        """Get database connection"""
        return sqlite3.connect(self.db_path)
    
    def insert_taxpayer(self, taxpayer: Taxpayer) -> bool:
        """Insert a taxpayer into the database"""
        try:
            conn = self.get_connection()
            cursor = conn.cursor()
            cursor.execute('''
                INSERT OR REPLACE INTO taxpayers 
                (pin_no, tax_payer_name, phone_no, station_name)
                VALUES (?, ?, ?, ?)
            ''', (taxpayer.pin_no, taxpayer.tax_payer_name, 
                  taxpayer.phone_no, taxpayer.station_name))
            conn.commit()
            conn.close()
            return True
        except Exception as e:
            print(f"Error inserting taxpayer: {e}")
            return False
    
    def insert_tax_record(self, record: TaxRecord) -> bool:  # Renamed from insert_obligation
        """Insert a tax record into the database"""
        try:
            conn = self.get_connection()
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO tax_records 
                (pin_no, obligation_name, from_date, to_date, principal, 
                 penalty, interest, grand_total, status)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (record._pin_no, record._obligation_name, 
                  record._from_date, record._to_date,
                  record._principal, record._penalty,
                  record._interest, record._grand_total,
                  record._status))
            conn.commit()
            conn.close()
            return True
        except Exception as e:
            print(f"Error inserting tax record: {e}")
            return False
    
    def get_taxpayer_records(self, pin_no: str) -> List[Dict]:  # Renamed from get_taxpayer_obligations
        """Get all records for a specific taxpayer"""
        conn = self.get_connection()
        query = '''
            SELECT r.*, t.tax_payer_name, t.phone_no 
            FROM tax_records r
            JOIN taxpayers t ON r.pin_no = t.pin_no
            WHERE r.pin_no = ?
            ORDER BY r.from_date DESC
        '''
        df = pd.read_sql_query(query, conn, params=(pin_no,))
        conn.close()
        return df.to_dict('records')
    
    def get_all_taxpayers(self) -> pd.DataFrame:
        """Get all taxpayers with their summary information"""
        conn = self.get_connection()
        query = '''
            SELECT 
                t.pin_no,
                t.tax_payer_name,
                t.phone_no,
                t.station_name,
                COUNT(r.record_id) as total_records,
                SUM(r.principal) as total_principal,
                SUM(r.penalty) as total_penalty,
                SUM(r.interest) as total_interest,
                SUM(r.grand_total) as total_arrears,
                SUM(CASE WHEN r.status = 'PENDING' THEN 1 ELSE 0 END) as pending_count,
                SUM(CASE WHEN r.status = 'PENALTY_APPLIED' THEN 1 ELSE 0 END) as penalty_count
            FROM taxpayers t
            LEFT JOIN tax_records r ON t.pin_no = r.pin_no
            GROUP BY t.pin_no
        '''
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    
    def get_summary_statistics(self) -> Dict:
        """Get summary statistics for the dashboard"""
        conn = self.get_connection()
        
        stats = {}
        
        # Total taxpayers
        stats['total_taxpayers'] = pd.read_sql_query(
            "SELECT COUNT(*) as count FROM taxpayers", conn
        ).iloc[0]['count']
        
        # Total records
        stats['total_records'] = pd.read_sql_query(
            "SELECT COUNT(*) as count FROM tax_records", conn
        ).iloc[0]['count']
        
        # Total arrears
        total_arrears = pd.read_sql_query(
            "SELECT SUM(grand_total) as total FROM tax_records", conn
        ).iloc[0]['total']
        stats['total_arrears'] = total_arrears if total_arrears else 0
        
        # Total penalties
        total_penalties = pd.read_sql_query(
            "SELECT SUM(penalty) as total FROM tax_records", conn
        ).iloc[0]['total']
        stats['total_penalties'] = total_penalties if total_penalties else 0
        
        # Total interest
        total_interest = pd.read_sql_query(
            "SELECT SUM(interest) as total FROM tax_records", conn
        ).iloc[0]['total']
        stats['total_interest'] = total_interest if total_interest else 0
        
        # By station
        stats['by_station'] = pd.read_sql_query('''
            SELECT station_name, COUNT(*) as count, SUM(r.grand_total) as arrears
            FROM taxpayers t
            JOIN tax_records r ON t.pin_no = r.pin_no
            GROUP BY station_name
        ''', conn)
        
        # By obligation type
        stats['by_obligation'] = pd.read_sql_query('''
            SELECT obligation_name, COUNT(*) as count, SUM(grand_total) as arrears
            FROM tax_records
            GROUP BY obligation_name
        ''', conn)
        
        conn.close()
        return stats


class DataImporter:
    """Handles importing data from CSV"""
    
    @staticmethod
    def clean_value(value):
        """Clean CSV values by removing quotes and commas"""
        if pd.isna(value):
            return None
        value = str(value).strip()
        value = value.replace('"', '').replace(',', '')
        if value == '-' or value == '':
            return None
        return value
    
    @staticmethod
    def import_from_csv(file_path: str, db_manager: DatabaseManager) -> Dict:
        """Import data from CSV file to database"""
        try:
            # Read CSV file
            df = pd.read_csv(file_path)
            
            stats = {'taxpayers_added': 0, 'records_added': 0, 'errors': 0}
            processed_pins = set()
            
            # Process each row
            for _, row in df.iterrows():
                try:
                    # Clean data
                    pin_no = DataImporter.clean_value(row.get('pin', ''))
                    if not pin_no:
                        continue
                    
                    # Create or update taxpayer
                    if pin_no not in processed_pins:
                        taxpayer = Taxpayer(
                            pin_no=pin_no,
                            tax_payer_name=DataImporter.clean_value(row.get('tax_payer_name', '')),
                            phone_no=DataImporter.clean_value(row.get('phone_no', '')),
                            station_name=DataImporter.clean_value(row.get('station_name', ''))
                        )
                        
                        if db_manager.insert_taxpayer(taxpayer):
                            stats['taxpayers_added'] += 1
                            processed_pins.add(pin_no)
                    
                    # Create tax record
                    record = TaxRecord(
                        pin_no=pin_no,
                        obligation_name=DataImporter.clean_value(row.get('obligation_name', '')),
                        from_date=DataImporter.clean_value(row.get('from_date', '')),
                        to_date=DataImporter.clean_value(row.get('to_date', '')),
                        principal=DataImporter.clean_value(row.get('PRINCIPAL', 0)),
                        penalty=DataImporter.clean_value(row.get('PENALTY', 0)),
                        interest=DataImporter.clean_value(row.get('INTEREST', 0))
                    )
                    
                    if db_manager.insert_tax_record(record):
                        stats['records_added'] += 1
                        
                except Exception as e:
                    stats['errors'] += 1
                    print(f"Error processing row: {e}")
            
            return stats
            
        except Exception as e:
            return {'error': str(e)}


# ============================================
# STREAMLIT APPLICATION
# ============================================

def init_session_state():
    """Initialize session state variables"""
    if 'authenticated' not in st.session_state:
        st.session_state.authenticated = False
    if 'username' not in st.session_state:
        st.session_state.username = None
    if 'role' not in st.session_state:
        st.session_state.role = 'VIEWER'


def login_page():
    """Login page for the application"""
    st.title("🔐 Tax Arrears Management System - Login")
    
    col1, col2, col3 = st.columns([1, 2, 1])
    
    with col2:
        st.markdown("""
        <div style='text-align: center; padding: 20px;'>
            <h2>Welcome to TAMS</h2>
            <p>Please login to access the system</p>
        </div>
        """, unsafe_allow_html=True)
        
        username = st.text_input("Username")
        password = st.text_input("Password", type="password")
        
        if st.button("Login", use_container_width=True):
            # Simple authentication (in production, use proper authentication)
            if username == "admin" and password == "admin123":
                st.session_state.authenticated = True
                st.session_state.username = username
                st.session_state.role = "ADMIN"
                st.success("Login successful!")
                st.rerun()
            elif username == "viewer" and password == "viewer123":
                st.session_state.authenticated = True
                st.session_state.username = username
                st.session_state.role = "VIEWER"
                st.success("Login successful!")
                st.rerun()
            else:
                st.error("Invalid username or password")


def dashboard_page(db_manager):
    """Main dashboard page"""
    st.title("📊 Tax Arrears Dashboard")
    
    # Get statistics
    stats = db_manager.get_summary_statistics()
    
    # Display KPI cards
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric("Total Taxpayers", f"{stats['total_taxpayers']:,}")
    
    with col2:
        st.metric("Total Records", f"{stats['total_records']:,}")  # Renamed from total_obligations
    
    with col3:
        st.metric("Total Arrears", f"KES {stats['total_arrears']:,.0f}")
    
    with col4:
        st.metric("Total Penalties", f"KES {stats['total_penalties']:,.0f}")
    
    # Charts
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Arrears by Station")
        if not stats['by_station'].empty:
            fig = px.pie(stats['by_station'], values='arrears', names='station_name',
                         title='Distribution of Tax Arrears by Station')
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No data available for station-wise analysis")
    
    with col2:
        st.subheader("Records by Obligation Type")  # Renamed
        if not stats['by_obligation'].empty:
            fig = px.bar(stats['by_obligation'].head(10), 
                         x='obligation_name', y='count',
                         title='Top 10 Obligation Types',
                         labels={'count': 'Number of Records',  # Renamed
                                'obligation_name': 'Obligation Type'})
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No data available for obligation type analysis")
    
    # Recent taxpayers
    st.subheader("Recent Taxpayers")
    df_taxpayers = db_manager.get_all_taxpayers()
    if not df_taxpayers.empty:
        st.dataframe(
            df_taxpayers[['pin_no', 'tax_payer_name', 'station_name', 
                          'total_records', 'total_principal', 'total_arrears']].head(10),  # Renamed
            use_container_width=True,
            hide_index=True
        )


def taxpayers_page(db_manager):
    """Taxpayers management page"""
    st.title("👥 Taxpayers Management")
    
    # Get all taxpayers
    df = db_manager.get_all_taxpayers()
    
    if df.empty:
        st.info("No taxpayers found in the database")
        return
    
    # Search and filter
    col1, col2 = st.columns(2)
    with col1:
        search_term = st.text_input("🔍 Search by name or PIN", "")
    
    with col2:
        station_filter = st.multiselect("Filter by Station", 
                                        options=df['station_name'].unique())
    
    # Apply filters
    filtered_df = df.copy()
    if search_term:
        filtered_df = filtered_df[
            filtered_df['tax_payer_name'].str.contains(search_term, case=False, na=False) |
            filtered_df['pin_no'].str.contains(search_term, case=False, na=False)
        ]
    
    if station_filter:
        filtered_df = filtered_df[filtered_df['station_name'].isin(station_filter)]
    
    # Display
    st.dataframe(
        filtered_df[['pin_no', 'tax_payer_name', 'station_name', 'phone_no',
                    'total_records', 'total_principal', 'total_penalty',
                    'total_interest', 'total_arrears']],  # Renamed
        use_container_width=True,
        hide_index=True
    )
    
    # Export option
    if st.button("📥 Export to CSV"):
        csv = filtered_df.to_csv(index=False)
        st.download_button(
            label="Download CSV",
            data=csv,
            file_name=f"taxpayers_{datetime.now().strftime('%Y%m%d')}.csv",
            mime="text/csv"
        )


def records_page(db_manager):  # Renamed from obligations_page
    """Tax records management page"""
    st.title("📋 Tax Records")
    
    conn = db_manager.get_connection()
    
    # Get all records with taxpayer details
    query = '''
        SELECT 
            r.record_id,
            t.pin_no,
            t.tax_payer_name,
            r.obligation_name,
            r.from_date,
            r.to_date,
            r.principal,
            r.penalty,
            r.interest,
            r.grand_total,
            r.status
        FROM tax_records r
        JOIN taxpayers t ON r.pin_no = t.pin_no
        ORDER BY r.from_date DESC
    '''
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    if df.empty:
        st.info("No tax records found in the database")
        return
    
    # Filters
    col1, col2, col3 = st.columns(3)
    
    with col1:
        status_filter = st.multiselect("Filter by Status", 
                                      options=df['status'].unique())
    
    with col2:
        name_filter = st.text_input("Search by taxpayer name", "")
    
    with col3:
        min_amount = st.number_input("Minimum amount", min_value=0, value=0, step=1000)
    
    # Apply filters
    filtered_df = df.copy()
    if status_filter:
        filtered_df = filtered_df[filtered_df['status'].isin(status_filter)]
    
    if name_filter:
        filtered_df = filtered_df[filtered_df['tax_payer_name'].str.contains(name_filter, case=False, na=False)]
    
    if min_amount > 0:
        filtered_df = filtered_df[filtered_df['grand_total'] >= min_amount]
    
    # Display
    st.dataframe(
        filtered_df,
        use_container_width=True,
        hide_index=True
    )
    
    # Summary
    st.subheader("Summary")
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        st.metric("Total Records", len(filtered_df))  # Renamed
    with col2:
        st.metric("Total Principal", f"KES {filtered_df['principal'].sum():,.0f}")
    with col3:
        st.metric("Total Penalties", f"KES {filtered_df['penalty'].sum():,.0f}")
    with col4:
        st.metric("Total Interest", f"KES {filtered_df['interest'].sum():,.0f}")


def analytics_page(db_manager):
    """Advanced analytics page"""
    st.title("📈 Advanced Analytics")
    
    conn = db_manager.get_connection()
    
    # Time series analysis
    query = '''
        SELECT 
            strftime('%Y', from_date) as year,
            COUNT(*) as record_count,
            SUM(principal) as total_principal,
            SUM(penalty) as total_penalty,
            SUM(interest) as total_interest,
            SUM(grand_total) as total_arrears
        FROM tax_records
        WHERE from_date IS NOT NULL AND from_date != ''
        GROUP BY year
        ORDER BY year
    '''
    df_yearly = pd.read_sql_query(query, conn)
    
    if not df_yearly.empty:
        col1, col2 = st.columns(2)
        
        with col1:
            fig = px.line(df_yearly, x='year', y='total_principal',
                          title='Annual Tax Principal Trend',
                          labels={'total_principal': 'Total Principal (KES)', 'year': 'Year'})
            st.plotly_chart(fig, use_container_width=True)
        
        with col2:
            fig = px.bar(df_yearly, x='year', y='record_count',
                         title='Annual Record Count',
                         labels={'record_count': 'Number of Records', 'year': 'Year'})
            st.plotly_chart(fig, use_container_width=True)
    
    # Top defaulters
    query = '''
        SELECT 
            t.tax_payer_name,
            t.pin_no,
            COUNT(r.record_id) as records,
            SUM(r.grand_total) as total_arrears,
            SUM(r.penalty) as total_penalties
        FROM taxpayers t
        JOIN tax_records r ON t.pin_no = r.pin_no
        WHERE r.grand_total > 0
        GROUP BY t.pin_no
        ORDER BY total_arrears DESC
        LIMIT 10
    '''
    df_top = pd.read_sql_query(query, conn)
    
    st.subheader("Top 10 Defaulters")
    if not df_top.empty:
        fig = px.bar(df_top, x='tax_payer_name', y='total_arrears',
                     title='Top 10 Taxpayers by Arrears Amount',
                     labels={'total_arrears': 'Total Arrears (KES)', 'tax_payer_name': 'Taxpayer'})
        st.plotly_chart(fig, use_container_width=True)
    
    # Status distribution
    query = '''
        SELECT 
            status,
            COUNT(*) as count,
            SUM(grand_total) as total_amount
        FROM tax_records
        GROUP BY status
    '''
    df_status = pd.read_sql_query(query, conn)
    
    if not df_status.empty:
        col1, col2 = st.columns(2)
        
        with col1:
            fig = px.pie(df_status, values='count', names='status',
                         title='Records by Status')  # Renamed
            st.plotly_chart(fig, use_container_width=True)
        
        with col2:
            fig = px.pie(df_status, values='total_amount', names='status',
                         title='Arrears Amount by Status')
            st.plotly_chart(fig, use_container_width=True)
    
    # Penalty vs Interest Analysis
    query = '''
        SELECT 
            obligation_name,
            SUM(penalty) as total_penalty,
            SUM(interest) as total_interest
        FROM tax_records
        GROUP BY obligation_name
        HAVING total_penalty > 0 OR total_interest > 0
        ORDER BY (total_penalty + total_interest) DESC
        LIMIT 10
    '''
    df_penalty_interest = pd.read_sql_query(query, conn)
    
    if not df_penalty_interest.empty:
        st.subheader("Top 10 Obligations with Penalties and Interest")
        fig = px.bar(df_penalty_interest, x='obligation_name', 
                     y=['total_penalty', 'total_interest'],
                     title='Penalty vs Interest by Obligation Type',
                     barmode='group')
        st.plotly_chart(fig, use_container_width=True)
    
    conn.close()


def import_data_page(db_manager):
    """Data import page"""
    st.title("📤 Import Data")
    
    st.info("Upload a CSV file with tax data to import into the system")
    
    uploaded_file = st.file_uploader("Choose a CSV file", type=['csv'])
    
    if uploaded_file is not None:
        # Show preview of uploaded data
        try:
            df_preview = pd.read_csv(uploaded_file)
            st.subheader("Data Preview")
            st.dataframe(df_preview.head(10), use_container_width=True)
            
            # Reset file pointer
            uploaded_file.seek(0)
            
            if st.button("Start Import", type="primary"):
                # Save uploaded file temporarily
                with open("temp_upload.csv", "wb") as f:
                    f.write(uploaded_file.getbuffer())
                
                with st.spinner("Importing data..."):
                    importer = DataImporter()
                    result = importer.import_from_csv("temp_upload.csv", db_manager)
                    
                    if 'error' in result:
                        st.error(f"Error importing data: {result['error']}")
                    else:
                        st.success("✅ Import completed successfully!")
                        col1, col2, col3 = st.columns(3)
                        col1.metric("Taxpayers Added", result['taxpayers_added'])
                        col2.metric("Records Added", result['records_added'])  # Renamed
                        col3.metric("Errors", result['errors'])
        except Exception as e:
            st.error(f"Error reading file: {e}")


def main():
    """Main application entry point"""
    st.set_page_config(
        page_title="Tax Arrears Management System",
        page_icon="💰",
        layout="wide",
        initial_sidebar_state="expanded"
    )
    
    # Initialize database
    create_database()
    
    # Initialize session state
    init_session_state()
    
    # Initialize database manager
    db_manager = DatabaseManager()
    
    # Check authentication
    if not st.session_state.authenticated:
        login_page()
        return
    
    # Sidebar navigation
    with st.sidebar:
        st.image("https://img.icons8.com/color/96/000000/tax.png", width=80)
        st.title(f"Welcome, {st.session_state.username}")
        st.markdown(f"**Role:** {st.session_state.role}")
        st.markdown("---")
        
        # Navigation menu
        menu_options = {
            "🏠 Dashboard": dashboard_page,
            "👥 Taxpayers": taxpayers_page,
            "📋 Tax Records": records_page,  # Renamed
            "📈 Analytics": analytics_page,
        }
        
        # Add admin-only pages
        if st.session_state.role == "ADMIN":
            menu_options["📤 Import Data"] = import_data_page
        
        selection = st.radio("Navigation", list(menu_options.keys()))
        
        st.markdown("---")
        if st.button("🚪 Logout", use_container_width=True):
            st.session_state.authenticated = False
            st.session_state.username = None
            st.session_state.role = None
            st.rerun()
    
    # Display selected page
    menu_options[selection](db_manager)


if __name__ == "__main__":
    main()

2026-02-14 08:59:43.394 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-14 08:59:43.415 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-14 08:59:43.416 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-14 08:59:43.417 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-14 08:59:43.418 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-14 08:59:43.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-14 08:59:43.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-14 08:59:43.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Database created successfully!
